In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
from environment import *

import setting

path_dict = path(setting)

CCAL version 0.7.8 @ /home/kwatme/miniconda3/lib/python3.6/site-packages/ccal/__init__.py


Created directory /home/kwatme/model_and_infer__rna_ccle_large_intestine/output/signal/nmf/7.
Created directory /home/kwatme/model_and_infer__rna_ccle_large_intestine/output/signal/nmf/7/wt.
Created directory /home/kwatme/model_and_infer__rna_ccle_large_intestine/output/signal/nmf/7/wt/match.
Created directory /home/kwatme/model_and_infer__rna_ccle_large_intestine/output/signal/nmf/7/wt/map.
Created directory /home/kwatme/model_and_infer__rna_ccle_large_intestine/output/signal/nmf/7/wt/hcc.
Created directory /home/kwatme/model_and_infer__rna_ccle_large_intestine/output/signal/nmf/7/wt/hcc/7.
Created directory /home/kwatme/model_and_infer__rna_ccle_large_intestine/output/signal/nmf/7/wt/hcc/7/match.
Created directory /home/kwatme/model_and_infer__rna_ccle_large_intestine/output/signal/nmf/7/h.
Created directory /home/kwatme/model_and_infer__rna_ccle_large_intestine/output/signal/nmf/7/h/match.
Created directory /home/kwatme/model_and_infer__rna_ccle_large_intestine/output/signal/nmf/7/h

In [3]:
w = pd.read_table(
    path_dict['w_file_path'],
    index_col=0,
)

h = pd.read_table(
    path_dict['h_file_path'],
    index_col=0,
)

In [4]:
from numpy import absolute, where
from scipy.spatial.distance import correlation


def compute_correlation_distance(
        x,
        y,
):

    correlation_distance = correlation(
        x,
        y,
    )

    return where(
        absolute(correlation_distance) < 1e-8,
        0,
        correlation_distance,
    )

In [ ]:
for wt_or_h, df, bandwidth_factor, element_marker_size in (
    (
        'wt',
        w.T,
        setting.GPS_MAP_WT_BANDWIDTH_FACTOR,
        setting.GPS_MAP_WT_ELEMENT_MARKER_SIZE,
    ),
    (
        'h',
        h,
        setting.GPS_MAP_H_BANDWIDTH_FACTOR,
        setting.GPS_MAP_H_ELEMENT_MARKER_SIZE,
    ),
):
    
    distance__column_x_column, k_hcc_distance_linkage, k_hcc_distance_linkage_ccc, hcc__k_x_column = ccal.hierarchical_consensus_cluster_with_multiple_k(
        df,
        setting.HCC_KS,
        n_job=setting.MAX_N_JOB,
        distance_function=compute_correlation_distance,
        plot_df=False,
        directory_path=path_dict['{}_hcc_directory_path'.format(wt_or_h)],
    )

    for k, element_cluster in hcc__k_x_column.iterrows(): 
        
        if wt_or_h == 'wt':
            
            yaxis_title = 'Feature'
        
        elif wt_or_h == 'h':
            
            yaxis_title = 'Sample'

        ccal.plot_heat_map(
            df,
            normalization_axis=0,
            normalization_method='-0-',
            column_annotation=element_cluster,
            title=k,
            xaxis_title=yaxis_title,
            yaxis_title='Part',
        )

Computing column-x-column distance with <function compute_correlation_distance at 0x7fecc472f0d0> ...
